In [1]:
!pip install torchsummary
import torch
import torch.nn as nn
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<img src="https://blog.kakaocdn.net/dn/YO3KB/btq0UcZPrx0/E0AAZXzLBtoYN5LXlVnk41/img.png"/>

<img src="https://blog.kakaocdn.net/dn/tQa0X/btq0NwlobsU/nmQxYObcdKfJZ49X3APnp0/img.png"/>

In [15]:
class DepthwiseSeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.separable = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = 1, padding = 0, bias = False)
        )

    def forward(self, x):
        x = self.separable(x)
        return x

In [27]:
class EntryFlow(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 0, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.conv2_residual = nn.Sequential(
            DepthwiseSeparableConv2d(64, 128),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            DepthwiseSeparableConv2d(128, 128),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )

        self.conv2_shortcut = nn.Sequential(
            nn.Conv2d(64, 128, 1, stride = 2, padding = 0),
            nn.BatchNorm2d(128),
        )

        self.conv3_residual = nn.Sequential(
            nn.ReLU(),
            DepthwiseSeparableConv2d(128, 256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            DepthwiseSeparableConv2d(256, 256),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )

        self.conv3_shortcut = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = 1, stride = 2, padding = 0),
            nn.BatchNorm2d(256)
        )

        self.conv4_residual = nn.Sequential(
            nn.ReLU(),
            DepthwiseSeparableConv2d(256, 728),
            nn.BatchNorm2d(728),
            nn.ReLU(),
            DepthwiseSeparableConv2d(728, 728),
            nn.BatchNorm2d(728),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )

        self.conv4_shortcut = nn.Sequential(
            nn.Conv2d(256, 728, 1, stride =2 , padding = 0),
            nn.BatchNorm2d(728)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2_residual(x) + self.conv2_shortcut(x)
        x = self.conv3_residual(x) + self.conv3_shortcut(x)
        x = self.conv4_residual(x) + self.conv4_shortcut(x)
        return x

In [28]:
class MiddleFlow(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_residual = nn.Sequential(
            nn.ReLU(),
            DepthwiseSeparableConv2d(728, 728),
            nn.BatchNorm2d(728),
            nn.ReLU(),
            DepthwiseSeparableConv2d(728, 728),
            nn.BatchNorm2d(728),
            nn.ReLU(),
            DepthwiseSeparableConv2d(728, 728),
            nn.BatchNorm2d(728),
        )

        self.conv_shortcut = nn.Sequential()

    def forward(self, x):
        return self.conv_shortcut(x) + self.conv_residual(x)

In [29]:
class ExitFlow(nn.Module):
    def __init__(self, num_classes = 1000):
        super().__init__()

        self.conv1_residual = nn.Sequential(
            nn.ReLU(),
            DepthwiseSeparableConv2d(728, 1024),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            DepthwiseSeparableConv2d(1024, 1024),
            nn.BatchNorm2d(1024),
            nn.MaxPool2d(kernel_size = 3, stride = 2 , padding = 1)
        )

        self.conv1_shortcut = nn.Sequential(
            nn.Conv2d(728, 1024, kernel_size = 1, stride= 2, padding = 0),
            nn.BatchNorm2d(1024),
        )

        self.conv2_residual = nn.Sequential(
            DepthwiseSeparableConv2d(1024, 1536),
            nn.BatchNorm2d(1536),
            nn.ReLU(),
            DepthwiseSeparableConv2d(1536, 2048),
            nn.BatchNorm2d(2048),
            nn.ReLU(),
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    
    def forward(self, x):
        x = self.conv1_residual(x) + self.conv1_shortcut(x)
        x = self.conv2_residual(x)
        x = self.avg_pool(x)

        return x

In [30]:
class XceptionNet(nn.Module):
    def __init__(self, num_classes = 10, init_weights = True):
        super().__init__()
        self.init_weights = init_weights

        self.entry = EntryFlow()
        self.middle = self._make_MiddleFlow()
        self.exit = ExitFlow()

        self.linear = nn.Linear(2048, num_classes)

        if self.init_weights:
            pass

    def forward(self, x):
        x = self.entry(x)
        x = self.middle(x)
        x = self.exit(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x 
    
    def _make_MiddleFlow(self):
        middle = nn.Sequential()
        for i in range(8):
            middle.add_module('middle_block_{}'.format(i), MiddleFlow())
        return middle
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init_kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init_constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init_constant_(m.weight, 1)
                nn.init_bias_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init_normal_(m.weight, 0, 0.01)
                nn.init_constant_(m.bias, 0)

In [32]:
x = torch.randn(3, 299, 299).unsqueeze(0)
model = XceptionNet()
output = model(x)
output.size()

torch.Size([1, 10])

In [33]:
summary(model, (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 150, 150]             864
       BatchNorm2d-2         [-1, 32, 150, 150]              64
              ReLU-3         [-1, 32, 150, 150]               0
            Conv2d-4         [-1, 64, 148, 148]          18,432
       BatchNorm2d-5         [-1, 64, 148, 148]             128
              ReLU-6         [-1, 64, 148, 148]               0
            Conv2d-7         [-1, 64, 148, 148]          36,864
            Conv2d-8        [-1, 128, 148, 148]           8,192
DepthwiseSeparableConv2d-9        [-1, 128, 148, 148]               0
      BatchNorm2d-10        [-1, 128, 148, 148]             256
             ReLU-11        [-1, 128, 148, 148]               0
           Conv2d-12        [-1, 128, 148, 148]         147,456
           Conv2d-13        [-1, 128, 148, 148]          16,384
DepthwiseSeparableConv2d-14      